In [1]:
%gui wx
%matplotlib inline

In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from mayavi import mlab
import seaborn as sns
from vtk_rw import read_vtk, write_vtk
import operator

In [3]:
def make_cmap(c):
    cmap = np.asarray(sns.color_palette('cubehelix', c-1))
    cmap = np.concatenate((np.array([[0.4,0.4,0.4]]), cmap), axis=0)
    cmap = np.concatenate((cmap, np.ones((c,1))), axis=1)
    cmap_seaborn = [tuple(cmap[i]) for i in range(len(cmap))]

    cmap_255=np.zeros_like(cmap)
    for row in range(cmap.shape[0]):
        cmap_255[row]=[np.floor(i * 255) for i in cmap[row]]
    cmap_255=cmap_255.astype(int)
    
    return cmap_seaborn, cmap_255

In [ ]:
def chebapprox(profiles, degree):
    profiles=np.array(profiles)
    cheb_coeffs=np.zeros((profiles.shape[0],degree+1))
    cheb_polynoms=np.zeros((profiles.shape[0],profiles.shape[1]))
    for c in range(profiles.shape[0]):
        x=np.array(range(profiles.shape[1]))
        y=profiles[c]
        cheb_coeffs[c]=np.polynomial.chebyshev.chebfit(x, y, degree)
        cheb_polynoms[c]=np.polynomial.chebyshev.chebval(x, cheb_coeffs[c])
    return cheb_coeffs, cheb_polynoms

### load data

In [ ]:
hemi='rh'
embed=3
k=5
pal, pal_255 = make_cmap(k+1)

mesh_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/surfs/lowres_%s_d.vtk'%hemi
mask_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/masks/%s_mask.1D.roi'%hemi
embed_file='/scr/ilz3/myelinconnect/all_data_on_simple_surf/clust/smooth_3/avg/%s_embed_%s.csv'  %(hemi, str(embed))
kmeans_file='/scr/ilz3/myelinconnect/all_data_on_simple_surf/clust/smooth_3/avg/%s_kmeans_%s_embed_%s.csv'%(hemi, str(k), str(embed))
t1_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/t1/avg_%s_profiles.npy'%(hemi)

#var_file=data_dir+'/results_k%s_s%s_other/%s_%s_cluster_var.csv'%(str(n_components_kmeans),smooth, sub, hemi)
#t_file=data_dir+'/results_k%s_s%s_other/%s_%s_cluster_avg_t.csv'%(str(n_components_kmeans),smooth, sub, hemi)
#p_file=data_dir+'/results_k%s_s%s_other/%s_%s_cluster_avg_p.csv'%(str(n_components_kmeans),smooth, sub, hemi)
#fig_file=data_dir+'/results_k%s_s%s_other/%s_%s_cluster_violins.svg'%(str(n_components_kmeans),smooth, sub, hemi)

In [ ]:
v,f,d = read_vtk(mesh_file)
x=v[:,0]
y=v[:,1]
z=v[:,2]
triangles=f


mask = np.loadtxt(mask_file)[:,0]
embed=np.loadtxt(embed_file, delimiter=',')
clust=np.loadtxt(kmeans_file, delimiter=',')
t1=np.load(t1_file)

### plot cluster / embedding

In [ ]:
mlab.figure(bgcolor=(1, 1, 1))
img = mlab.triangular_mesh(x,y,z,triangles,scalars=embed[:,1], colormap='RdBu')
#lut = img.module_manager.scalar_lut_manager.lut.table.to_array()
#img.module_manager.scalar_lut_manager.lut.table = pal_255
mlab.draw()
mlab.show()

### Get T1 values in all clusters

In [ ]:
# make a list of dictionaries for each cluster k0, k1, ...(0=mask to max kmeans) 
t1_clust={}
for c in range(int(clust.max()+1)):
    t1_clust['k'+str(c)]=[]

# write all t1 profiles in one cluster into the list of its dictionary
for i in range(len(t1)):
    k=int(clust[i])
    t1_clust['k'+str(k)].append(t1[i])
    
# make other dictionaries with 3 to 7 intracortical profiles and averages, thresholded  1400 to 2500 mean
t1_clust_37={}
t1_clust_37_avg={}
t1_clust_37_coeff_0={}
t1_clust_37_coeff_1={}
t1_clust_37_coeff_2={}
t1_clust_37_coeff_3={}
for c in range(int(clust.max()+1)):
    t1_clust_37['k'+str(c)]=[pro[3:8] for pro in t1_clust['k'+str(c)]
                            if np.mean(pro[3:8]) < 2500
                            and np.mean(pro[3:8]) > 1500]
    t1_clust_37_avg['k'+str(c)]=[np.mean(pro_37) for pro_37 in t1_clust_37['k'+str(c)]]
    
    cheb_coeffs, cheb_polynoms = chebapprox(t1_clust_37['k'+str(c)], 10)
    
    t1_clust_37_coeff_0['k'+str(c)]=[cheb_coeffs[j][0] for j in range(len(t1_clust_37['k'+str(c)]))]
    t1_clust_37_coeff_1['k'+str(c)]=[cheb_coeffs[j][1] for j in range(len(t1_clust_37['k'+str(c)]))]
    t1_clust_37_coeff_2['k'+str(c)]=[cheb_coeffs[j][2] for j in range(len(t1_clust_37['k'+str(c)]))]
    t1_clust_37_coeff_3['k'+str(c)]=[cheb_coeffs[j][3] for j in range(len(t1_clust_37['k'+str(c)]))]


### Compare variance of kmeans cluster vs all other clusters and mean between all clusters

In [ ]:
T = t1_clust_37_avg

In [ ]:
levene_array=np.zeros((6,int(clust.max())))
col=[]
sigk=[]
for c1 in range(int(clust.max())):
    a=T['k'+str(c1+1)]
    b=[]
    for c2 in range(int(clust.max())):
        if (c2+1) != (c1+1):
            b+=T['k'+str(c2+1)]
    W,p=stats.levene(a, b)
    col.append(int(c1+1))
    levene_array[0,c1]=W
    levene_array[1,c1]=p
    levene_array[2,c1]=np.var(a)
    levene_array[3,c1]=np.var(b)
    if p <(0.05/int(clust.max())):
        levene_array[4,c1]=True
    else:
        levene_array[4,c1]=False
    
    if np.var(a)<np.var(b):
        levene_array[5,c1]=True
    else:
        levene_array[5,c1]=False
        
    if (levene_array[4,c1]==True) and (levene_array[5,c1]==True):
        sigk.append(c1+1)

levene_df=pd.DataFrame(levene_array, columns=col, index=['W', 'p', 'var k', 'var other', 'p<0.05/k', 'var decrease'])
#levene_df.to_csv(var_file)


t_array=np.zeros((int(clust.max()),int(clust.max())))
p_array=np.zeros((int(clust.max()),int(clust.max())))
col2=[]
for c1 in range(int(clust.max())):
    for c2 in range(int(clust.max())):
        if c2>=c1:
            a=T['k'+str(c1+1)]
            b=T['k'+str(c2+1)]
            t,p=stats.ttest_ind(a, b)
            t_array[c1][c2]=t
            p_array[c1][c2]=p
    col2.append(c1+1)
t_df=pd.DataFrame(t_array, columns=col2, index=col2)
p_df=pd.DataFrame(p_array, columns=col2, index=col2)

#t_df.to_csv(t_file)
#p_df.to_csv(p_file)

### Plot clustering, T1 values, Levene's statistics and T statistics

In [ ]:
means = {}
for c in range(int(clust.max())):
    means['k'+str(c+1)] = np.mean(T['k'+str(c+1)])
sorted_means = sorted(means.items(), key=operator.itemgetter(1))
sorted_pal = [pal[int(sorted_means[p][0][1])] for p in range(len(sorted_means))] 

plot_list=[]
cluster_list=[]
for m in range(len(sorted_means)):
    plot_list+=T[sorted_means[m][0]]
    cluster_list+=len(T[sorted_means[m][0]])*[sorted_means[m][0]]
    
plot_df=pd.DataFrame(columns=['t1', 'cluster'])
plot_df['t1']=plot_list
plot_df['cluster']=cluster_list

In [ ]:
sns.palplot(sorted_pal)

In [ ]:
sns.set_context('notebook', font_scale=1.8)
fig = plt.figure(figsize=(10,10))
sns.violinplot(x='cluster', y='t1',data=plot_df, palette=sorted_pal, saturation=1, inner='box')
sns.axlabel('','', fontsize=22)
#savefig(fig_file)

Variance comparison

In [ ]:
pd.options.display.float_format = '{:,.2e}'.format
levene_df

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
levene_df.tail(2)

In [ ]:
sig=np.zeros_like(clust)
for i in sigk: 
    sig[clust == i] = i

In [ ]:
sigk

In [ ]:
allcort=np.where(clust!=0)[0].shape[0]
sigcort=np.where(sig!=0)[0].shape[0]
sigpart=sigcort/allcort
print float(sigpart)

In [ ]:
mlab.figure(bgcolor=(1, 1, 1))
img = mlab.triangular_mesh(x,y,z,triangles,scalars=sig, colormap='Set2')
lut = img.module_manager.scalar_lut_manager.lut.table.to_array()
img.module_manager.scalar_lut_manager.lut.table = pal_255
mlab.draw()
mlab.show()

T test (T values, p values)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
t_df

In [ ]:
pd.options.display.float_format = '{:,.2e}'.format
p_df